In [1]:
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

<h3>Get the input location where the user would like to go out</h3>

In [2]:
address = input('Enter the location:')


geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of the City of '+ address + ' are {}, {}.'.format(latitude, longitude))

Enter the location:Velachery
The geograpical coordinate of the City of Velachery are 12.98016545, 80.2228505622558.


/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:4: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.


In [3]:
CLIENT_ID = '######'
# your Foursquare ID
CLIENT_SECRET = '#######'

 # your Foursquare Secret
VERSION = '20180605' # Foursquare API version


<h3>The below code extracts the category of the place from the json result returned by FourSquare API</h3>

In [4]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

<h3>Gets the distance withing which the user prefer to travel and find out the restaurants within the given distance range</h3>

In [5]:
radius=input('Enter the distance within which you prefer to find a restaurant in metres')
LIMIT=100
url = 'https://api.foursquare.com/v2/venues/search?categoryId=4d4b7105d754a06374d81259&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude, 
    longitude, 
    radius, 
    LIMIT
)
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
results = requests.get(url).json()
results

Enter the distance within which you prefer to find a restaurant in metres7000


{'meta': {'code': 200, 'requestId': '5d98330dbcbf7a00395932ff'},
 'response': {'venues': [{'id': '531f0ea9498e4d4369274e16',
    'name': 'Haagen Dazs, Phoenix Market City',
    'location': {'address': 'Ground Floor, Phoenix Market City',
     'crossStreet': 'Velacherry Main Road',
     'lat': 12.992135690803618,
     'lng': 80.21703434061988,
     'labeledLatLngs': [{'label': 'display',
       'lat': 12.992135690803618,
       'lng': 80.21703434061988}],
     'distance': 1474,
     'cc': 'IN',
     'city': 'Chennai',
     'state': 'Tamil Nadu',
     'country': 'India',
     'formattedAddress': ['Ground Floor, Phoenix Market City (Velacherry Main Road)',
      'Chennai',
      'Tamil Nadu',
      'India']},
    'categories': [{'id': '4bf58dd8d48988d1c9941735',
      'name': 'Ice Cream Shop',
      'pluralName': 'Ice Cream Shops',
      'shortName': 'Ice Cream',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/icecream_',
       'suffix': '.png'},
      'primary': Tr

In [6]:
venues = results['response']['venues']
venues
nearby_venues = json_normalize(venues) # flatten JSON

filtered_columns = ['name','id','categories','location.distance','location.lat', 'location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues
    


,name,id,categories,distance,lat,lng
0,"Haagen Dazs, Phoenix Market City",531f0ea9498e4d4369274e16,Ice Cream Shop,1474,12.992136,80.217034
1,Adyar Ananda Bhavan,5d91e896d47a7c000885ef75,Restaurant,3351,13.008609,80.212720
2,Starbucks,5b9e10ae1ffe97002cc752a7,Coffee Shop,6965,13.041685,80.234580
3,Starbucks,55f99e33498e4ddd66a2210e,Coffee Shop,6135,13.029225,80.248628
4,Hotel Saravana Bhavan,4c6ba1d669b4ef3b01ec454e,South Indian Restaurant,7922,13.050776,80.213741
5,Paradise,5783684a498ee3d0fefe4701,Indian Restaurant,3059,12.972837,80.250035
6,Krispy Kreme,5620507f498e2200145bf07a,Donut Shop,6130,12.983939,80.166464
7,Sangeetha Veg Restaurant,4cf12fd888de370468b3702b,Vegetarian / Vegan Restaurant,6302,13.025631,80.257484
8,Dindigul Thalapakatti,4ba62336f964a5202d3639e3,South Indian Restaurant,6187,13.033716,80.238134
9,Bommaraja Mess,5d67f4b6307914000863252f,Chettinad Restaurant,9060,12.898909,80.227690


<h3>Displays the different categories/cuisines of restaurants available within the given distance range</h3>

In [7]:
nearby_venues_grouped = nearby_venues.groupby('categories').size().reset_index(name='counts')
category_restaurants=nearby_venues_grouped.categories.unique()
nearby_venues_grouped
print('The categories of restaurants present in the area are listed below ')
for elem in category_restaurants:
        print (elem) 

The categories of restaurants present in the area are listed below 
Asian Restaurant
BBQ Joint
Bakery
Café
Chettinad Restaurant
Coffee Shop
Dessert Shop
Donut Shop
Hotel
Ice Cream Shop
Indian Restaurant
Japanese Restaurant
Juice Bar
Modern European Restaurant
Office
Pet Café
Restaurant
Seafood Restaurant
Snack Place
South Indian Restaurant
Vegetarian / Vegan Restaurant


In [8]:
def get_rating(row):
    try:
        categories_list = row['response']['venue']['rating']
    except:
        categories_list = 0
    #print(categories_list)
    if categories_list == 0:
        return 0
    else:
        return row['response']['venue']['rating']

<h3>User can enter the category.cuisine that they want to try out in the city and get the available restaurants for that cuisine within the given distance<h3>

In [9]:
req_category=input('Enter the cuisine/category of restaurant that you prefer to go')
import pandas as pd
import numpy as np
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
req_cuisine_restaurants=nearby_venues.loc[nearby_venues['categories'] == req_category]
i=0

import pandas as pdi
restaurant_info = pdi.DataFrame(columns=['Name', 'Category','Price_Tier','Rating','Distance','Latitude','Longitude'])
for row in req_cuisine_restaurants.iterrows():    
    venue_id=row[1].id
    url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id,CLIENT_ID,CLIENT_SECRET,VERSION)
    results = requests.get(url).json()
    #print(results)
    restaurant_info.loc[i, 'Name'] = results['response']['venue']['name']
    restaurant_info.loc[i, 'Category'] = results['response']['venue']['categories'][0]['name']
    restaurant_info.loc[i,'Price_Tier']=results['response']['venue']['price']['tier']
    restaurant_info.loc[i, 'Rating']=get_rating(results)
    restaurant_info.loc[i, 'Distance']=row[1].distance
    restaurant_info.loc[i, 'Latitude']=results['response']['venue']['location']['lat']
    restaurant_info.loc[i, 'Longitude']=results['response']['venue']['location']['lng']
    #restaurant_info.loc[i, 'Rating'] = results['response']['venue']['rating']
    i=i+1



Enter the cuisine/category of restaurant that you prefer to goIndian Restaurant


<h3>Restaurants for the selected cuisine sorted by customer ratings </h3>

In [10]:
restaurant_info.sort_values('Rating',ascending=False)

,Name,Category,Price_Tier,Rating,Distance,Latitude,Longitude
1,Peshawri,Indian Restaurant,2,8.7,3392,13.0105,80.2203
3,Murugan Idli Shop,Indian Restaurant,2,8,7071,13.0425,80.2355
0,Paradise,Indian Restaurant,2,7.9,3059,12.9728,80.25
5,Food village,Indian Restaurant,2,7.7,6617,12.9279,80.2518
7,Murugan Idli Shop,Indian Restaurant,2,7.5,7831,13.0483,80.205
2,Saravana Bhavan,Indian Restaurant,2,7.4,6436,13.0365,80.2362
4,Namma Veedu Vasantha Bhavan,Indian Restaurant,2,5.3,5008,12.945,80.2517
6,Hotel City Gate,Indian Restaurant,2,5.1,3662,12.9879,80.2557


<h3>Restaurants for the selected cuisine sorted by Distance </h3>

In [11]:
restaurant_info.sort_values('Distance')

,Name,Category,Price_Tier,Rating,Distance,Latitude,Longitude
0,Paradise,Indian Restaurant,2,7.9,3059,12.9728,80.25
1,Peshawri,Indian Restaurant,2,8.7,3392,13.0105,80.2203
6,Hotel City Gate,Indian Restaurant,2,5.1,3662,12.9879,80.2557
4,Namma Veedu Vasantha Bhavan,Indian Restaurant,2,5.3,5008,12.945,80.2517
2,Saravana Bhavan,Indian Restaurant,2,7.4,6436,13.0365,80.2362
5,Food village,Indian Restaurant,2,7.7,6617,12.9279,80.2518
3,Murugan Idli Shop,Indian Restaurant,2,8,7071,13.0425,80.2355
7,Murugan Idli Shop,Indian Restaurant,2,7.5,7831,13.0483,80.205


<h3>Map showing the restaurants</h3>

In [16]:
#!conda install -c conda-forge folium=0.5.0
import folium
mapit = None
mapit = folium.Map(location=[latitude,longitude], zoom_start=12)
for row in restaurant_info.iterrows():
    folium.Marker( location=[row[1].Latitude,row[1].Longitude]).add_to(mapit)
    

mapit